In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Nov 16 11:38:54 2018

@author: stelios
"""

import os, sys
lib_path = os.path.abspath(os.path.join('..'))+"/adan"
sys.path.append(lib_path)
lib_path = os.path.abspath(os.path.join('..'))
sys.path.append(lib_path)

import adan
from adan.aiem.genetics import *
from adan.aidc.utilities import *
from adan.aidc.feature_selection import *
#from adan.aipm.optimizers.xgboost_wrapper import *
#from adan.aipm.optimizers.optimizers import *
#from adan.aipm.optimizers.xgbOptimizer import *

from adan.metrics.metrics_utilities import *
from adan.aipm.optimizers.hyperparam_optimization import *

from adan.aiem.symbolic_modelling import *
from adan.aist.mappers import *
from adan.aidc.utilities import *
#from adan.aipm.optimizers import *
from adan.aidc.utilities import *
from adan.aiem.genetics.genetic_programming import *
#from adan.modellingDeprecated.estimation_utilities import *
#from adan.aipm.optimizers.xgbOptimizer import *
from matplotlib import pyplot as plt

#np.random.seed(19115)


print('Community crimes')
#https://www.kaggle.com/kkanda/analyzing-uci-crime-and-communities-dataset/data

target_name='ViolentCrimesPerPop'
path="community crimes/crimedata.csv"
task='regression'
ngen=100

df=readData(path)
#df.drop(['communityname','state','assaultPerPop','larcPerPop'],inplace=True,axis=1)
df.drop(['communityname','state','murdPerPop','rapesPerPop','robbbPerPop','assaultPerPop',
         'rapes','assaults','robberies','murders','autoTheft','autoTheftPerPop','racePctWhite','autoTheftPerPop','burglPerPop'],inplace=True,axis=1)
#this is where the cleaning takes place
#this function
df2, target,scaler,centerer,categorical_vars,filler,log=prepareTrainData(df,target_name,center=False)
#feature selection
df2=chooseBest(df2,target,limit_n=1000,method="num",quant=0.15)[0]
g=findFeaturesGP(df=df2,target=target,ngen=ngen,max_tree=4,population=3000,
                 features=3,n_processes=1,evaluator=evalPearsonCorNumba, allowed_time=None)

k = findSymbolicExpression(df2,target,g,scaler,task=task,
                 features_type='best_features_plus_cols')

perfs=[]
for c in k:
    print(c[1])
    perfs.append(c[1])
perfs=np.array(perfs)

final_models=[]
for m in k:
    final_models.append(Model(m[0],m[1],g,scaler,centerer,categorical_vars,filler,task=task))

model = final_models[np.where(perfs==max(perfs))[0][0]]

df=df[df[target_name].notnull()]
df3=prepareTestData(df,scaler,centerer,categorical_vars_match=categorical_vars,
                    filler=filler,model=model.model)
paok = pd.read_csv('features_test.csv')

res_train = model.evaluate(df)
res_train[res_train<0]=0
#res_train=final_models[3].evaluate(df)
error = np.mean(np.abs(res_train-df[target_name]))
print(error)
plt.close()
plt.scatter(res_train,df[target_name])
plt.xlabel('predicted')
plt.ylabel('true values')

res_train = model.evaluate(df)
error = np.mean(np.abs(res_train-target))
print(error)

realizer=sentenceRealizerSymbolic()
realizer.interpretSymbolic(k,task='regression')
res=realizer.realizeAll()
print(res)
#get best variables
print(g['variables'])


variables=model.return_variables()
print('the important variables are :'+str(variables))




/Users/stelios/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/Users/stelios/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/stelios/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Thi

Community crimes


/Users/stelios/Dropbox/ADAN/adan_scikitlearn0p20/adan2/adan/adan/aidc/utilities.py:39: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  df=pd.read_csv(path,sep=sep,na_values=na_values,header=header,encoding=encoding)
/Users/stelios/Dropbox/ADAN/adan_scikitlearn0p20/adan2/adan/adan/functions.py:28: RuntimeWarning: divide by zero encountered in true_divide
  res=np.array(left * 1.0 / right)
/Users/stelios/Dropbox/ADAN/adan_scikitlearn0p20/adan2/adan/adan/functions.py:28: RuntimeWarning: invalid value encountered in true_divide
  res=np.array(left * 1.0 / right)
/Users/stelios/Dropbox/ADAN/adan_scikitlearn0p20/adan2/adan/adan/functions.py:37: RuntimeWarning: divide by zero encountered in true_divide
  return 1.0/x
/Users/stelios/Dropbox/ADAN/adan_scikitlearn0p20/adan2/adan/adan/functions.py:46: RuntimeWarning: invalid value encountered in sqrt
  res=n

   	      	                    fitness                     	              size             
   	      	------------------------------------------------	-------------------------------
gen	nevals	avg     	max     	min	std     	avg    	max	min	std    
0  	3000  	0.154345	0.744023	-2 	0.457148	4.10133	25 	2  	3.13811
1  	2676  	0.303388	0.747647	-2 	0.364944	3.39633	21 	1  	2.26861
2  	2703  	0.407598	0.780442	-2 	0.350323	3.07467	19 	1  	1.80825
3  	2731  	0.513024	0.755037	-2 	0.321043	2.73767	22 	1  	1.63651


<string>:1: RuntimeWarning: overflow encountered in square


...

76 	2721  	0.552482	0.753319	-2 	0.595859	2.90367	21 	1  	1.18366
77 	2717  	0.573648	0.757867	-2 	0.542546	2.933  	15 	1  	1.17012
78 	2751  	0.564306	0.757867	-2 	0.569529	2.93833	27 	1  	1.29558
79 	2753  	0.586799	0.757867	-2 	0.52255 	2.93433	17 	1  	1.19778
80 	2737  	0.576208	0.753324	-2 	0.539911	2.986  	26 	1  	1.43055
81 	2745  	0.569981	0.753324	-2 	0.554853	2.95067	21 	1  	1.28721
82 	2751  	0.558195	0.753319	-2 	0.57633 	2.976  	18 	1  	1.24904
83 	2746  	0.581019	0.753319	-2 	0.53019 	2.986  	24 	1  	1.39109
84 	2711  	0.578751	0.753319	-2 	0.543233	2.92833	19 	1  	1.1809 
85 	2767  	0.564928	0.753319	-2 	0.563174	2.96767	25 	1  	1.35202
86 	2763  	0.56169 	0.753319	-2 	0.567322	2.953  	24 	1  	1.35109
87 	2746  	0.548292	0.753319	-2 	0.592215	2.912  	16 	1  	1.17143
88 	2717  	0.544728	0.753319	-2 	0.595822	2.94033	23 	1  	1.25729
89 	2707  	0.559022	0.753319	-2 	0.572584	2.94233	23 	1  	1.25818
90 	2757  	0.535083	0.753319	-2 	0.605976	2.948  	19 	1  	1.25537
91 	2719  

/Users/stelios/Dropbox/ADAN/adan_scikitlearn0p20/adan2/adan/adan/functions.py:22: RuntimeWarning: invalid value encountered in subtract
  return x-y


100	2736  	0.461683	0.753319	-2 	0.654584	2.94867	21 	1  	1.41776


/Users/stelios/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/stelios/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/stelios/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/stelios/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:4

interpreting coefficients
finding acceptable solutions
(2943900.0*var_NumKidsBornNeverMar + 1744989.2*var_burglaries - 1162744300.0*var_larcenies + 792930.0*var_nonViolPerPop + 3444.0*sqrt(var_burglaries + var_nonViolPerPop) + 1159257600.0*log(var_larcenies) - 515.37885, 0.5124037503997761, array([ -27.274353,   68.84137 ,  665.17584 , ...,  613.56647 ,
        688.78033 , 1099.0293  ], dtype=float32), LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))
time required for length:0.00010585784912109375
time required for atoms:9.5367431640625e-07
(2943900.0*var_NumKidsBornNeverMar + 1744989.2*var_burglaries - 1162744300.0*var_larcenies + 792930.0*var_nonViolPerPop + 3444.0*sqrt(var_burglaries + var_nonViolPerPop) + 1159257600.0*log(var_larcenies) - 515.37885, 0.5124037503997761, array([ -27.274353,   68.84137 ,  665.17584 , ...,  613.56647 ,
        688.78033 , 1099.0293  ], dtype=float32), LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=Fal

In [2]:
import shap
from sklearn.tree import DecisionTreeRegressor
import pandas as pd

shap.initjs()

tree=DecisionTreeRegressor()
remaining_data=df2.loc[:,variables]
remaining_data.replace([np.inf, -np.inf], np.nan,inplace=True)
remaining_data.fillna(0,inplace=True)

res_train[res_train==np.inf]=max(target)
res_train[res_train==-np.inf]=0
res_train[np.isnan(res_train)]=0
tree.fit(remaining_data,res_train)

explainer = shap.TreeExplainer(tree)
shap_values = explainer.shap_values(remaining_data)

# plot the SHAP values for the Setosa output of the first instance
shap.force_plot(explainer.expected_value, shap_values, remaining_data)

In [3]:
shap.force_plot(explainer.expected_value, shap_values[0,:], remaining_data.iloc[0,:])


In [4]:
res_train


0        -27.296326
1         68.858459
2        665.281311
3          0.000000
5        748.105530
6        171.835999
7        526.453186
8        437.082092
9       1288.199219
10       880.128967
11      1134.585938
12       617.242249
13      1166.921875
16       366.263733
17       572.652405
18       328.704651
19       834.808655
21      2097.621094
22       644.230530
23       562.037170
24       665.769592
25       575.369202
26       343.724182
27       850.800842
28      1035.960938
29       555.286194
30       556.783752
31       707.199280
32         0.000000
33        12.962402
           ...     
2185    1253.183594
2186    1337.476562
2187     299.499084
2188     101.398987
2189     140.581116
2190     -40.398132
2191     446.954651
2192     411.113342
2193     729.878967
2194     406.785217
2195     230.613831
2196       0.000000
2197     145.462952
2198     364.715393
2199      57.141174
2200     176.895569
2201     631.293030
2202     540.927795
2203     699.528381
